In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
gap_train= pd.read_pickle('./temp_result/train_kaggle_processed_PPA_PCA_PPA_neither')
gap_test= pd.read_pickle('./temp_result/test_kaggle_processed_PPA_PCA_PPA_neither')
gap_valid= pd.read_pickle('./temp_result/valid_kaggle_processed_PPA_PCA_PPA_neither')

In [3]:
NUM_TRAIN = gap_train.count().values[0]
NUM_TEST = gap_test.count().values[0]
NUM_VALID = gap_valid.count().values[0]
def label(A,B):
    if A is True:
        return 0
    if B is True:
        return 1
    return 2
def switch_label(l):
    if l==2:
        return 2
    return 1-l

In [4]:
def process_prediction(pred):
    s = pred.shape[0]//2
    pred0 = pred[0:s,:]
    pred1 = pred[s:,:]
    pred1 = pred1[:,[1,0,2]]
    pred_out = pred0+pred1
    return pred_out/2

In [5]:
def switch_A_B(df):
    columnsTitles = ["B_dist","A_dist","B_pos", "A_pos","pron_pos", "B_vector", "A_vector","pron_vector","product_vector_B","product_vector_A","label"]
    df2=df.reindex(columns=columnsTitles).copy()
    df2.columns = df.columns
    df2.label = df2.label.map(switch_label)
    return pd.concat([df,df2],axis = 0, sort = False)

In [6]:
import torch
def compute_loss(sub_df,test_data):
    pred = torch.Tensor(np.log(sub_df.loc[:,['A','B','NEITHER']].values))
    label = torch.LongTensor(list(test_data.label))
    loss = torch.nn.NLLLoss()
    loss_value = loss(pred,label).item()
    return loss_value

In [7]:
def for_neither(x):
    if x.shape[0]!= 0:
        return np.mean(x,axis = 0)
    else:
        return np.zeros(256)

In [8]:
gap_train.neither_vector[25]

array([], shape=(0, 256), dtype=float32)

In [9]:
train_data = gap_train.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx',"neither_idx","name_list","neither_idx_2","name_list_2","neither_vector"])
train_data.A_vector = train_data.A_vector.map(lambda x:np.mean(x,axis = 0))
train_data.B_vector = train_data.B_vector.map(lambda x:np.mean(x,axis = 0))
train_data.pron_vector = train_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
train_data.neither_vector_2 = train_data.neither_vector_2.map(lambda x:for_neither(x))
train_data["product_vector_A"] = train_data.A_vector*train_data.pron_vector
train_data["product_vector_B"] = train_data.B_vector*train_data.pron_vector
train_data["product_vector_neither"] = train_data.neither_vector_2*train_data.pron_vector
train_data["label"] = train_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
train_data = train_data.drop(columns= ["A-coref","B-coref"])
#train_data = switch_A_B(train_data)

In [10]:
test_data = gap_test.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx',"neither_idx","name_list","neither_idx_2","name_list_2","neither_vector"])
test_data.A_vector = test_data.A_vector.map(lambda x:np.mean(x,axis = 0))
test_data.B_vector = test_data.B_vector.map(lambda x:np.mean(x,axis = 0))
test_data.pron_vector = test_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
test_data.neither_vector_2 = test_data.neither_vector_2.map(lambda x:for_neither(x))
test_data["product_vector_A"] = test_data.A_vector*test_data.pron_vector
test_data["product_vector_B"] = test_data.B_vector*test_data.pron_vector
test_data["product_vector_neither"] = test_data.neither_vector_2*test_data.pron_vector
test_data["label"] = test_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
test_data = test_data.drop(columns= ["A-coref","B-coref"])
#test_data = switch_A_B(test_data)

In [11]:
valid_data = gap_valid.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx',"neither_idx","name_list","neither_idx_2","name_list_2","neither_vector"])
valid_data.A_vector = valid_data.A_vector.map(lambda x:np.mean(x,axis = 0))
valid_data.B_vector = valid_data.B_vector.map(lambda x:np.mean(x,axis = 0))
valid_data.pron_vector = valid_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
valid_data.neither_vector_2 = valid_data.neither_vector_2.map(lambda x:for_neither(x))
valid_data["product_vector_A"] = valid_data.A_vector*valid_data.pron_vector
valid_data["product_vector_B"] = valid_data.B_vector*valid_data.pron_vector
valid_data["product_vector_neither"] = valid_data.neither_vector_2*valid_data.pron_vector
valid_data["label"] = valid_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
valid_data = valid_data.drop(columns= ["A-coref","B-coref"])
#valid_data = switch_A_B(valid_data)

In [12]:
valid_data.head()

,A_dist,B_dist,A_pos,B_pos,pron_pos,A_vector,B_vector,pron_vector,neither_vector_2,product_vector_A,product_vector_B,product_vector_neither,label
0,0.018,0.004,0.693548,0.806452,0.838710,"[0.20037952, 0.2805402, -0.11013528, -0.492183...","[0.6041415, -0.13538514, 0.15207358, -0.076126...","[0.8586822, -1.2192798, 0.09194927, -0.4327072...","[-0.25571027, -0.07773098, -0.48689944, -1.220...","[0.17206234, -0.342057, -0.010126858, 0.212971...","[0.51876557, 0.16507237, 0.013983054, 0.032940...","[-0.21957387, 0.09477582, -0.044770047, 0.5279...",2
1,0.030,0.012,0.511111,0.711111,0.844444,"[0.067682624, 0.42009318, -0.009991955, -0.659...","[-0.438681, 0.48596978, -0.02240046, 0.0839553...","[-0.67510414, -0.39920956, 0.07794422, 0.43378...","[-0.12306414, -0.019772578, -0.148793, -0.4155...","[-0.04569282, -0.16770521, -0.0007788151, -0.2...","[0.29615536, -0.19400378, -0.0017459863, 0.036...","[0.08308111, 0.007893402, -0.011597554, -0.180...",1
2,0.022,0.010,0.405128,0.435897,0.461538,"[-0.40055174, -0.06318157, 0.08668772, -0.0661...","[0.67819536, 0.21544528, -0.39938855, 0.618967...","[0.72358626, -1.0747175, 0.050702423, -0.89089...","[0.15148854, 0.1907654, -0.17670102, -0.126718...","[-0.28983372, 0.06790234, 0.0043952777, 0.0588...","[0.49073285, -0.23154281, -0.020249967, -0.551...","[0.10961503, -0.20501891, -0.00895917, 0.11289...",1
3,0.012,0.004,0.201521,0.216730,0.224335,"[-0.33478984, -0.63129133, -0.5514479, 0.59391...","[0.50680375, 0.71950006, -0.25772676, 0.375783...","[1.2850071, 0.049930945, -0.26918668, 0.253453...","[0.41241223, 0.8078015, 0.50750864, 0.03485240...","[-0.43020734, -0.031520974, 0.14844243, 0.1505...","[0.6512464, 0.035925318, 0.06937661, 0.0952435...","[0.52995265, 0.04033429, -0.13661456, 0.008833...",0
4,0.056,0.042,0.321839,0.362069,0.482759,"[0.25996214, -0.15495634, -0.5617872, -0.04960...","[0.2925861, 0.2669702, 0.03382411, -0.3578499,...","[-0.87992877, -0.07835998, 0.12989467, -0.2304...","[0.28072405, 0.48110938, -0.024320625, -0.4141...","[-0.22874817, 0.012142375, -0.07297316, 0.0114...","[-0.2574549, -0.020919777, 0.0043935715, 0.082...","[-0.24701716, -0.037699718, -0.0031591195, 0.0...",1


In [13]:
columns = train_data.columns[:-1]
X_train = np.concatenate([np.array(list(train_data[col])).reshape(train_data.shape[0],-1) for col in columns],axis = 1)
y_train = list(train_data.label)   

In [14]:
X_valid = np.concatenate([np.array(list(valid_data[col])).reshape(valid_data.shape[0],-1) for col in columns],axis = 1)
y_valid = list(valid_data.label)   

In [15]:
X_test = np.concatenate([np.array(list(test_data[col])).reshape(test_data.shape[0],-1) for col in columns],axis = 1)
y_test= list(test_data.label)   

In [16]:
import pickle
f = open( "./temp_result/base_model_data", "wb" )
pickle.dump(X_train,  f)
pickle.dump(y_train,  f)
pickle.dump(X_valid,  f)
pickle.dump(y_valid,  f)
pickle.dump(X_test,  f)
pickle.dump(y_test,  f)
f.close()

In [17]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial',C = 0.02).fit(X_train, y_train)

/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [18]:
pred_lr = lr.predict_proba(X_test)

In [19]:
#pred_lr = process_prediction(pred_lr)
sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_lr


sub_df.to_csv("./test_and_submit/submission+model+lr@"+str(datetime.datetime.now())+".csv", index=False)

sub_df.head()


,ID,A,B,NEITHER
0,development-1,0.648500,0.243299,0.108201
1,development-2,0.792850,0.142018,0.065131
2,development-3,0.045926,0.932941,0.021133
3,development-4,0.050576,0.759220,0.190203
4,development-5,0.017313,0.911611,0.071076


In [20]:
compute_loss(sub_df,test_data)

0.5355990529060364

In [21]:
pred_lr_train = lr.predict_proba(X_train)

In [22]:
import pickle
f = open( "./temp_result/lr_result", "wb" )
pickle.dump(pred_lr_train,  f)
pickle.dump(pred_lr,  f)
f.close()

In [23]:
from sklearn.svm import SVC
svm = SVC(C = 7.0,verbose=True,probability = True,gamma = "auto",class_weight='balanced').fit(X_train, y_train)

[LibSVM]

In [24]:
pred_svm = svm.predict_proba(X_test)

In [25]:
#pred_svm = process_prediction(pred_svm)
sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_svm

In [26]:
sub_df.to_csv("./test_and_submit/submission+model+svm@"+str(datetime.datetime.now())+".csv", index=False)
sub_df.head()

,ID,A,B,NEITHER
0,development-1,0.708325,0.219804,0.071872
1,development-2,0.801409,0.130339,0.068252
2,development-3,0.028252,0.939505,0.032243
3,development-4,0.042207,0.743166,0.214628
4,development-5,0.018973,0.872788,0.108239


In [27]:
compute_loss(sub_df,test_data)

0.5287905931472778

In [28]:
pred_svm_train = svm.predict_proba(X_train)

In [29]:
import pickle
f = open( "./temp_result/svm_result", "wb" )
pickle.dump(pred_svm_train,  f)
pickle.dump(pred_svm,  f)
f.close()

In [30]:
#X_train.shape

In [31]:
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(1797, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.95),
            nn.Linear(128, 3)
        )
        nn.init.xavier_uniform_(self.layers[0].weight)
        nn.init.xavier_uniform_(self.layers[-1].weight)
    def forward(self, x):
        # convert tensor (128, 1, 28, 28) --> (128, 1*28*28)
        x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x

In [32]:
EPOCHS = 500
batch_size = 25
mlp = MLP()
mlp.cuda()
opt = torch.optim.Adam(mlp.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()#weight = torch.Tensor([1.0,1.0,5.0]))
loss_fn.cuda()

CrossEntropyLoss()

In [33]:
for e in range(EPOCHS):
    for b in range(0,X_train.shape[0],batch_size):
        mlp.train()
        batch_data = X_train[b:b+batch_size,:]
        batch_label = y_train[b:b+batch_size]
        output = mlp(torch.Tensor(batch_data).cuda())
        batch_label = torch.LongTensor(batch_label).cuda()
        loss = loss_fn(output,batch_label)
        l2_norm = torch.norm(mlp.layers[-1].weight, p=2)
        loss += l2_norm*0.09
        l2_norm = torch.norm(mlp.layers[0].weight, p=2)
        loss += l2_norm*0.03
        opt.zero_grad()
        loss.backward()
        opt.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(e + 1, EPOCHS, loss.item()))

epoch [1/500], loss:2.2593
epoch [2/500], loss:1.3828
epoch [3/500], loss:1.3110
epoch [4/500], loss:1.2690
epoch [5/500], loss:1.2776
epoch [6/500], loss:1.2781
epoch [7/500], loss:1.3250
epoch [8/500], loss:1.1108
epoch [9/500], loss:1.0807
epoch [10/500], loss:1.0815
epoch [11/500], loss:1.0329
epoch [12/500], loss:1.0915
epoch [13/500], loss:0.9155
epoch [14/500], loss:1.1203
epoch [15/500], loss:0.9897
epoch [16/500], loss:0.9564
epoch [17/500], loss:0.9643
epoch [18/500], loss:0.9062
epoch [19/500], loss:0.9481
epoch [20/500], loss:1.0303
epoch [21/500], loss:1.0023
epoch [22/500], loss:0.9352
epoch [23/500], loss:0.9819
epoch [24/500], loss:1.0583
epoch [25/500], loss:0.8532
epoch [26/500], loss:0.9305
epoch [27/500], loss:0.8078
epoch [28/500], loss:0.8622
epoch [29/500], loss:0.8998
epoch [30/500], loss:0.8562
epoch [31/500], loss:0.7669
epoch [32/500], loss:0.8521
epoch [33/500], loss:0.9868
epoch [34/500], loss:0.8308
epoch [35/500], loss:0.7436
epoch [36/500], loss:0.9850
e

epoch [289/500], loss:0.9418
epoch [290/500], loss:0.9568
epoch [291/500], loss:0.8981
epoch [292/500], loss:0.8141
epoch [293/500], loss:0.9330
epoch [294/500], loss:0.7256
epoch [295/500], loss:0.8271
epoch [296/500], loss:0.8947
epoch [297/500], loss:0.9807
epoch [298/500], loss:0.9346
epoch [299/500], loss:0.9394
epoch [300/500], loss:0.9471
epoch [301/500], loss:0.8795
epoch [302/500], loss:0.9478
epoch [303/500], loss:0.9960
epoch [304/500], loss:0.8462
epoch [305/500], loss:0.8689
epoch [306/500], loss:1.1023
epoch [307/500], loss:0.9656
epoch [308/500], loss:1.0978
epoch [309/500], loss:0.9976
epoch [310/500], loss:0.8809
epoch [311/500], loss:0.8406
epoch [312/500], loss:0.8487
epoch [313/500], loss:0.8777
epoch [314/500], loss:0.8479
epoch [315/500], loss:0.9001
epoch [316/500], loss:0.8964
epoch [317/500], loss:0.8930
epoch [318/500], loss:0.9417
epoch [319/500], loss:0.8804
epoch [320/500], loss:0.8252
epoch [321/500], loss:0.9746
epoch [322/500], loss:0.9319
epoch [323/500

In [34]:
mlp.eval()
pred_mlp = torch.nn.Softmax(dim = 1)(mlp(torch.Tensor(X_test[:,:]).cuda())).cpu().data.numpy()
pred_mlp_train = torch.nn.Softmax(dim = 1)(mlp(torch.Tensor(X_train[:,:]).cuda())).cpu().data.numpy()

In [35]:
#pred_mlp = process_prediction(pred_mlp)

In [36]:
sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_mlp
sub_df.to_csv("./test_and_submit/submission+model+mlp@"+str(datetime.datetime.now())+".csv", index=False)
sub_df.head(7)

,ID,A,B,NEITHER
0,development-1,0.496124,0.332154,0.171722
1,development-2,0.896544,0.063167,0.040289
2,development-3,0.069529,0.900002,0.030469
3,development-4,0.106871,0.713545,0.179584
4,development-5,0.009373,0.976605,0.014022
5,development-6,0.996813,0.001552,0.001635
6,development-7,0.742837,0.118898,0.138265


In [37]:
compute_loss(sub_df,test_data)

0.5617367625236511

In [38]:
import pickle
f = open( "./temp_result/mlp_result", "wb" )
pickle.dump(pred_mlp_train,  f)
pickle.dump(pred_mlp,  f)
f.close()

In [39]:
import pickle
f = open( "./temp_result/label", "wb" )
pickle.dump(y_train,  f)
pickle.dump(y_test,  f)
f.close()